In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pandas as pd

c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
product_df = pd.read_csv('C:/Users/ACER/Downloads/Amazon-Products.csv')
product_name = product_df['name'].tolist()

In [4]:
count = product_df['name'].str.endswith('...', na=False).sum()
print(count)

122226


In [15]:
print(len(product_df))

551585


In [9]:

batch_size = 1000
embedding_file = "../data/embeddings.npy"
num_products = len(product_name)
# Generate embeddings in batches and save them
all_embeddings = []
for i in range(0, num_products, batch_size):
    batch_texts = product_name[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts).astype('float32')
    all_embeddings.append(batch_embeddings)

all_embeddings = np.vstack(all_embeddings)
np.save(embedding_file, all_embeddings)


In [10]:
all_embeddings = np.load(embedding_file)
dimension = all_embeddings.shape[1]

nlist = 100  
quantizer = faiss.IndexFlatL2(dimension) 
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

index.train(all_embeddings)

batch_size = 10000
for i in range(0, all_embeddings.shape[0], batch_size):
    batch_embeddings = all_embeddings[i:i+batch_size]
    index.add(batch_embeddings)
    
# Save the index to disk
faiss.write_index(index, "../data/large_index.ivf")

In [6]:
index = faiss.read_index("../model/large_index.ivf")

# Query example
query = "Apple iPhone 13 (256GB)"
query_embedding = model.encode([query]).astype('float32')

# Perform similarity search
k = 20  # Top 10 results
distances, indices = index.search(query_embedding, k)

# Display results
print("Query:", query)
for i in range(k):
    print(f"Result {i+1}:")
    print(f"  Text: {product_name[indices[0][i]]}")
    print(f"  Distance: {distances[0][i]}")

Query: Apple iPhone 13 (256GB)
Result 1:
  Text: Apple iPhone 13 (256GB) - Midnight
  Distance: 0.3968679904937744
Result 2:
  Text: Apple iPhone 13 (128GB) - Midnight
  Distance: 0.3995436728000641
Result 3:
  Text: Apple iPhone 13 (256GB) - Blue
  Distance: 0.5627176761627197
Result 4:
  Text: Apple iPhone 13 (128GB) - Blue
  Distance: 0.5786619186401367
Result 5:
  Text: Apple iPhone 13 (128GB) - Blue
  Distance: 0.5786619186401367
Result 6:
  Text: Apple iPhone 14 Plus (256 GB) - Blue
  Distance: 0.6229243278503418
Result 7:
  Text: Apple iPhone 14 Pro (256 GB) - Space Black
  Distance: 0.6652201414108276
Result 8:
  Text: Apple iPhone 14 Pro Max (128 GB) - Space Black
  Distance: 0.6759583950042725
Result 9:
  Text: Apple iPhone 12 (64GB) - Black
  Distance: 0.6826390624046326
Result 10:
  Text: Apple iPhone 14 Pro Max (256 GB) - Deep Purple
  Distance: 0.6864816546440125
Result 11:
  Text: Apple iPhone 12 (128GB) - Black
  Distance: 0.698520839214325
Result 12:
  Text: Apple iPho